In [1]:
# import os
# os.environ['CIMGRAPH_TIME'] = '1'
# from cimgraph.utils.timing import print_timing, clear_timing
# print_timing()
# clear_timing()

In [1]:
import cimgraph.data_profile.cimhub_2023 as cim
from uuid import UUID
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import BlazegraphConnection
from cimgraph.models import FeederModel
from cimgraph.models import DistributedArea
import cimgraph.queries.sparql as sparql
import cimgraph.utils as utils
from defusedxml.ElementTree import parse
cim_profile = 'cimhub_2023'

In [2]:
from mermaid import Mermaid

In [3]:


from cimgraph.databases.fileparsers import XMLFile
params = ConnectionParameters(filename="./test_models/ieee13.xml", cim_profile=cim_profile, iec61970_301=8)
xml_file = XMLFile(params)
feeder = cim.Feeder()
network = FeederModel(connection=xml_file, container=feeder, distributed=False)


In [4]:
# utils.write_xml(network, 'test_models/ieee13_out.xml', {'rdf':"http://www.w3.org/1999/02/22-rdf-syntax-ns#", 'cim': 'http://iec.ch/TC57/2013/CIM-schema-cim16#', 'eu': 'http://iec.ch/TC57/CIM100#', 'xsi:schemaLocation': 'http://iec.ch/TC57/2013/CIM-schema-cim16# CIM16.xsd'})

In [5]:
obj = list(network.graph[cim.ACLineSegment].values())[0]
obj.pprint()

{
    "@id": "a04cdfb1-e951-4fc4-8882-0323cd70ae3c",
    "@type": "ACLineSegment",
    "name": "650632",
    "Location": {
        "@id": "3a63b621-3fa6-4805-96f9-dd57a7f88acf",
        "@type": "Location"
    },
    "EquipmentContainer": {
        "@id": "49ad8e07-3bf9-a4e2-cb8f-c3722f837b62",
        "@type": "Feeder"
    },
    "BaseVoltage": {
        "@id": "2a158e0c-cd01-4a50-aeba-59d761fcf15d",
        "@type": "BaseVoltage"
    },
    "Terminals": [
        {
            "@id": "000ba1f2-6879-47d7-826d-3cacd23ee20b",
            "@type": "Terminal"
        },
        {
            "@id": "3589a9be-f3be-45c3-b84e-3ce5b6105d11",
            "@type": "Terminal"
        }
    ],
    "length": "609.6",
    "ACLineSegmentPhases": [
        {
            "@id": "2a5a1025-ad81-4bfa-a7f0-ed70f7f43daa",
            "@type": "ACLineSegmentPhase"
        },
        {
            "@id": "2d60b241-de87-48bd-a8d3-fe4b705b40ed",
            "@type": "ACLineSegmentPhase"
        },
        {
  

In [6]:
from mermaid import Mermaid
from cimgraph import utils

Mermaid(utils.get_mermaid(cim.ACLineSegment))

In [7]:
Mermaid(utils.get_mermaid(cim.ACLineSegment,show_inherited=True))

In [ ]:
line = network.graph[cim.ACLineSegment][UUID('a04cdfb1-e951-4fc4-8882-0323cd70ae3c')]
diagram_text = utils.get_mermaid(line)
Mermaid(utils.get_mermaid(line))

In [10]:
breaker = cim.Breaker(name = 'breaker12', open=False, inService=True)
substation = cim.Substation(name = 'sub1234')
t1 = cim.Terminal(name = 'brk12_t1')
t2 = cim.Terminal(name = 'brk12_t2')
basev = cim.BaseVoltage(name = 'base12kv', nominalVoltage=12470)
breaker.EquipmentContainer = substation
breaker.Terminals = [t1, t2]
breaker.BaseVoltage = basev

Mermaid(utils.get_mermaid(breaker))

In [11]:
data_list = [cim.ACLineSegment(), cim.ACLineSegmentPhase(), cim.WireInfo()]
# len(set(map(type, data_list))) <= 1
# set(map(type, data_list)) == {type}
set(map(isinstance, data_list, [cim.Identity]*len(data_list))) == {True}



True

In [12]:
Mermaid(utils.get_mermaid([cim.ACLineSegment, cim.ACLineSegmentPhase, cim.WireInfo]))

In [17]:
import cimgraph.data_profile.cim17v40 as cim

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (cim17v40.py, line 65473)

In [14]:
import enum
from enum import Enum
def list_associations(cim_class, show_inherited:bool=False):
        indent = '    '
        parent_classes = list(cim_class.__mro__)
        parent_classes.pop(len(parent_classes) - 1)
        if type(cim_class) is not enum.EnumMeta and len(parent_classes)>1:
            answer = 'classDiagram\n'
            answer += indent + 'class ' + cim_class.__name__ + '{\n'

            for attribute in cim_class.__annotations__.keys():
                attr = cim_class.__dataclass_fields__[attribute]
                attr_type = str(attr.type)
                try:
                    if 'Attribute' in attr.metadata['type'] or 'Enumeration' in attr.metadata['type']:
                        edge = attr_type.split('[')[1].split(']')[0]

                        answer += f'{indent*2}+ {attribute} : {edge}\n'
                except:
                    pass

            if show_inherited:

                for parent in parent_classes:
                    if len(parent.__annotations__.keys()) > 0 and parent.__name__ != cim_class.__name__:

                        for attribute in parent.__annotations__.keys():
                            attr = parent.__dataclass_fields__[attribute]
                            attr_type = str(attr.type)
                            try:
                                if 'Attribute' in attr.metadata['type'] or 'Enumeration' in attr.metadata['type']:
                                    edge = attr_type.split('[')[1].split(']')[0]

                                    answer += f'{indent*2}+ {attribute} : {edge}\n'
                            except:
                                pass

            answer += indent + '}\n'

            answer += indent + f'{parent_classes[1].__name__} <|-- {cim_class.__name__} : inherits from\n'

            for attribute in cim_class.__annotations__.keys():
                attr = cim_class.__dataclass_fields__[attribute]
                attr_type = str(attr.type)
                try:
                    if 'Association' in attr.metadata['type'] or 'Of Aggregate' in attr.metadata['type']:
                        edge = attr_type.split('[')[1].split(']')[0].replace('|', 'or')
                        if 'list' in attr_type:
                            answer += f'{indent}{cim_class.__name__} --> "many" {edge} : {attribute} \n'
                        else:
                            answer += f'{indent}{cim_class.__name__} --> "one" {edge} : {attribute} \n'
                    elif 'Aggregate Of' in attr.metadata['type']:
                        edge = attr_type.split('[')[1].split(']')[0].replace('|', 'or')
                        if 'list' in attr_type:
                            answer += f'{indent}{cim_class.__name__} --o "many" {edge} : {attribute} \n'
                        else:
                            answer += f'{indent}{cim_class.__name__} --o "one" {edge} : {attribute} \n'
                except:
                    pass

            if show_inherited:

                for parent in parent_classes:
                    if len(parent.__annotations__.keys()) > 0 and parent.__name__ != cim_class.__name__:

                        for attribute in parent.__annotations__.keys():
                            attr = parent.__dataclass_fields__[attribute]
                            attr_type = str(attr.type)

                            try:
                                if 'Association' in attr.metadata['type'] or 'Of Aggregate' in attr.metadata['type']:
                                    edge = attr_type.split('[')[1].split(']')[0].replace('|', 'or')
                                    if 'list' in attr_type:
                                        answer += f'{indent}{cim_class.__name__} --> "many" {edge} : {attribute} \n'
                                    else:
                                        answer += f'{indent}{cim_class.__name__} --> "one" {edge} : {attribute} \n'
                                elif 'Aggregate Of' in attr.metadata['type']:
                                    edge = attr_type.split('[')[1].split(']')[0].replace('|', 'or')
                                    if 'list' in attr_type:
                                        answer += f'{indent}{cim_class.__name__} --o "many" {edge} : {attribute} \n'
                                    else:
                                        answer += f'{indent}{cim_class.__name__} --o "one" {edge} : {attribute} \n'
                            except:
                                pass
        return answer

In [15]:
Mermaid(list_associations(cim.ACLineSegment, show_inherited=True))

In [ ]:
utils.write_json_ld(network, 'test_models/ieee13.jsonld')


In [ ]:
import re

def upper_camel_to_lower_snake(name):
    # Add an underscore before each uppercase letter (except the first one)
    # and convert the whole string to lowercase
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    s2 = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1)
    
    return s2.lower()

# Example usage
upper_camel_word = "UpperCamelCase"
lower_snake_case_word = upper_camel_to_lower_snake(upper_camel_word)
print(lower_snake_case_word)

In [ ]:
cim_class = cim.PhotovoltaicUnit
attribute = 'maxP'
question = f"Using the CIM-Graph python library, how do I write an API call to find the attribute {attribute} of a particular {cim_class.__name__} instance?"
answer = f"""To get the values of {cim_class.__name__}.{attribute}, you would call CIM-Graph as follows:
```python
{upper_camel_to_lower_snake(cim_class.__name__)} = network.graph[cim.{cim_class.__name__}][uuid]
result = {upper_camel_to_lower_snake(cim_class.__name__)}.{attribute}
```"""
import json
print(question)
print(json.dumps(answer))

In [ ]:

file = open('ieee13.csv', 'w')
for cim_class in network.graph:
    for cim_obj in network.graph[cim_class].values():
        question = f'Can you create an example of a CIM {cim_class.__name__} object with mRID {cim_obj.uri()}?'
        answer = cim_obj.__str__().replace(',', '\n')
        row = question + ','
        file.write()

In [ ]:
utils.write_xml(network, 'out13.xml')

In [ ]:
from cimgraph.databases.fileparsers import XMLFile
params2 = ConnectionParameters(filename="/home/ande188/CIM-Graph/tests/test_models/ieee13.xml", cim_profile=cim_profile, iec61970_301=8)
xml_file2 = XMLFile(params2)
feeder_mrid = "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
feeder2 = cim.Feeder(mRID = feeder_mrid)
network2 = FeederModel(connection=xml_file2, container=feeder2, distributed=False)